In [1]:
!jupyter nbconvert --execute toronto_neighborhoods_02.ipynb
import pandas as pd
import folium
import pickle

[NbConvertApp] Converting notebook toronto_neighborhoods_02.ipynb to html
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 258006 bytes to toronto_neighborhoods_02.html


In [55]:
from geopy.geocoders import Nominatim

import requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [2]:
details = pickle.load(open('details.dataframe.p','rb'))
details.sample(5)

,Postal Code,Borough,Neighborhoods,Latitude,Longitude
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
20,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
83,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
28,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259
37,M4E,East Toronto,The Beaches,43.676357,-79.293031


## Create empty map of Toronto using average latitude and longitude values

In [23]:
latitude = details['Latitude'].mean()
longitude = details['Longitude'].mean()

The mean coordinates of Toronto are: lat 43.70460773398059, lon -79.39715291165048


In [15]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

## add markers to map

In [17]:
for lat, lng, borough, neighborhood in zip(details['Latitude'], details['Longitude'], details['Borough'], details['Neighborhoods']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

## Display marked-up map

In [ ]:
map_toronto

# Segmenting and Clustering

In [41]:
downtown_details = details[details['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_details.head()

,Postal Code,Borough,Neighborhoods,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [42]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="to_on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of {address} are {latitude}, {longitude}.')

The geograpical coordinate of Downtown Toronto are 43.654027, -79.3802003.


### Create an empty map of Downtown Toronto

In [43]:
map_downtown = folium.Map(location=[latitude,longitude], zoom_start=12)

### add markers to map

In [44]:
for lat, lng, label in zip(downtown_details['Latitude'], downtown_details['Longitude'], downtown_details['Neighborhoods']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  


### Display map

In [ ]:
map_downtown

# Collecting venues within neighborhoods from Foursquare

In [46]:
CLIENT_ID = 'E4FR4UPF4F4AQ0RNYPER1CQSN1ZEWBDN0GOV5AVXDB5YGQ1I' # your Foursquare ID
CLIENT_SECRET = 'BMCRNSXV3PXCI0FKWXEVEHXKFVIR2VFUMZTM5OHQ4PJBPSCD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: E4FR4UPF4F4AQ0RNYPER1CQSN1ZEWBDN0GOV5AVXDB5YGQ1I
CLIENT_SECRET:BMCRNSXV3PXCI0FKWXEVEHXKFVIR2VFUMZTM5OHQ4PJBPSCD


In [106]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [113]:
from tqdm import tqdm

In [118]:
downtown_venues = pd.DataFrame()

for neighborhood_index, neighborhood_name in tqdm(enumerate(downtown_details['Neighborhoods'])):

    neighborhood_latitude, neighborhood_longitude = downtown_details.loc[neighborhood_index, ['Latitude','Longitude']]

    url=(f"https://api.foursquare.com/v2/venues/explore?"+
         f"&client_id={CLIENT_ID}"+
         f"&client_secret={CLIENT_SECRET}"+
         f"&v={VERSION}"+
         f"&ll={neighborhood_latitude},{neighborhood_longitude}"+
         f"&radius={500}"+
         f"&limit={100}")
    
    results = requests.get(url).json()
    assert results['meta']['code'] == 200
    
    venues = results['response']['groups'][0]['items']
    nearby_venues = json_normalize(venues) # flatten JSON
    
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    
    # label with neighborhood name
    nearby_venues['Neighborhood'] = neighborhood_name
    
    # Add nearby neighborhoods venues to dataframe of downtown venues
    downtown_venues = downtown_venues.append(nearby_venues, ignore_index=True)
    
    #print(nearby_venues.head())


18it [00:39,  1.79s/it]


## Create and display map of all collected venues in Downtown Toronto

In [121]:
map_downtown_venues = folium.Map(location = [downtown_venues.lat.mean(), downtown_venues.lng.mean()], zoom_start=10)
for lat, lng, label in zip(downtown_venues['lat'], downtown_venues['lng'], downtown_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_venues)  
map_downtown_venues